In [2]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib

In [3]:
company_list = []

for i in range(1,8):
    url = "https://job.incruit.com/jobdb_list/searchjob.asp?kw=%B7%CE%BA%BF&rgn1=149&page={page_number}"        # 자신의 찾으려는 웹사이트 입력
    req = Request(url.format(page_number = urllib.parse.quote(str(i))))
    res = urlopen(req)
    soup = BeautifulSoup(res, "html.parser")
    
    element = soup.find_all('ul',class_='c_row')
    for each in element:
        cl_top = each.find('div',class_='cl_top')
        cell_mid = each.find('div',class_='cell_mid')

        company = []
        name = each.find('a',class_='cpname').string    # 회사이름
        link = each.find('a',class_='cpname')['href']

        company.append(name)
        company.append(link)
        company_list.append(company)

In [4]:
company_list

[['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['(주)유비소시어스', 'https://www.incruit.com/company/24638930'],
 ['(주)엑사로보틱스', 'https://www.incruit.com/company/1684312614'],
 ['(주)잡뉴스솔로몬서치', 'https://www.incruit.com/company/11351847'],
 ['(주)투비파트너즈', 'https://www.incruit.com/company/1670139994'],
 ['(주)에바', 'https://www.incruit.com/company/1680817418'],
 ['(주)피플케어코리아', 'https://www.incruit.com/company/2226864'],
 ['(주)잡뉴스솔로몬서치', 'https://www.incruit.com/company/11351847'],
 ['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['(주)유진로봇', 'https://www.incruit.com/company/5487148'],
 ['위즈로봇학원', 'https://www.incruit.com/company/1681813847'],
 ['쓰리핸즈', 'https://www.incruit.com/company/1685388129'],
 ['(주)프로매치코리아', 'https://www.incruit.com/company/728244'],


In [5]:
import pandas as pd 
robot_df = pd.DataFrame(company_list, columns=['회사명', '링크'])

In [6]:
robot_df = robot_df[robot_df['링크'].str.contains('http')]

In [8]:
from tqdm import tqdm 
from user_agent import generate_user_agent
import time

info_list = []
r1_list = []
r2_list = []
r3_list = []


for idx, row in tqdm(robot_df.iterrows(), total=len(robot_df.index)):
    req = Request(url=row['링크'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')
    time.sleep(0.2)

    try:
        table =  soup.find(class_='table_li')
        info_list.append(table.get_text())
    except:
        info_list.append('-')     

    try:    
        rev = soup.find(class_='table_b sub_table')
        tbody = rev.find('tbody')
        r1 = tbody.find_all('tr')[0]
        r2 = tbody.find_all('tr')[1]
        r3 = tbody.find_all('tr')[2]
    
        r1_list.append(r1.get_text())
        r2_list.append(r2.get_text())
        r3_list.append(r3.get_text())

    except:
        r1_list.append('-')
        r2_list.append('-')
        r3_list.append('-')

100%|██████████| 253/253 [02:36<00:00,  1.62it/s]


In [10]:
robot_df['info'] = info_list
robot_df['r1'] = r1_list
robot_df['r2'] = r2_list
robot_df['r3'] = r3_list

In [11]:
robot_df['기업규모'] =  robot_df['info'].str.split('기업규모').str[1]
robot_df['매출액'] =  robot_df['기업규모'].str.split('매출액').str[1]
robot_df['평균연봉'] =  robot_df['매출액'].str.split('평균연봉').str[1]

In [12]:
robot_df['기업규모'] =  robot_df['기업규모'].str.split('대표자').str[0]
robot_df['매출액'] =  robot_df['매출액'].str.split('사원수').str[0]

In [13]:
robot_df['평균연봉'] =  robot_df['평균연봉'].str.split('자세히보기').str[0]

In [14]:
robot_df['자본총계1'] =  robot_df['r1'].str.split('\n').str[0]
robot_df['자본총계2'] =  robot_df['r2'].str.split('\n').str[0]
robot_df['자본총계3'] =  robot_df['r3'].str.split('\n').str[0]

robot_df['매출액1'] =  robot_df['r1'].str.split('\n').str[1]
robot_df['매출액2'] =  robot_df['r2'].str.split('\n').str[1]
robot_df['매출액3'] =  robot_df['r3'].str.split('\n').str[1]

robot_df['순이익1'] =  robot_df['r1'].str.split('\n').str[2]
robot_df['순이익2'] =  robot_df['r2'].str.split('\n').str[2]
robot_df['순이익3'] =  robot_df['r3'].str.split('\n').str[2]

In [15]:
robot_df.drop(columns=['링크','info','r1','r2','r3'], inplace=True)

In [16]:
import pandas as pd

# 예제 데이터프레임
data = {'금액': ['- 1,000억 5,000만원', '3,000억원', '500원', '10억 2,000만원', '6,848원', '1조 5,000억 3,000만원', '- 11억 578만원']}
df = pd.DataFrame(data)

def convert_currency_to_int(value):
    if isinstance(value, (float, int)):
        return int(value)

    result = 0
    is_negative = False

    # '억' 또는 '억원' 다음에 '원'이 오는 경우를 처리
    value = value.replace('억원', '억 원')

    for part in value.split():
        if '조' in part:
            result += int(part.replace('조', '').replace(',', '')) * 10**8
        elif '억' in part or '억원' in part:
            result += int(part.replace('억', '').replace('억원', '').replace(',', '')) * 10**4
        elif '만원' in part:
            result += int(part.replace('만원', '').replace(',', ''))
        elif '원' in part:
            result += 0
        elif '-' in part:
            is_negative = True

    return -result if is_negative else result

# '금액' 열에 함수 적용
df['금액'] = df['금액'].apply(convert_currency_to_int)

# 결과 표시
print(df)

          금액
0  -10005000
1   30000000
2          0
3     102000
4          0
5  150003000
6    -110578


In [17]:
robot_df['평균연봉'] = pd.to_numeric(robot_df['평균연봉'].str.replace(',', '').str.replace('만원', ''), errors='coerce').astype('Int64')

In [18]:
robot_df['매출액1'] = robot_df['매출액1'].str.replace('↑','')
robot_df['매출액2'] = robot_df['매출액2'].str.replace('↑','')
robot_df['매출액3'] = robot_df['매출액3'].str.replace('↑','')
robot_df['순이익1'] = robot_df['순이익1'].str.replace('↑','')
robot_df['순이익2'] = robot_df['순이익2'].str.replace('↑','')
robot_df['순이익3'] = robot_df['순이익3'].str.replace('↑','')

robot_df['매출액1'] = robot_df['매출액1'].str.replace('↓','')
robot_df['매출액2'] = robot_df['매출액2'].str.replace('↓','')
robot_df['매출액3'] = robot_df['매출액3'].str.replace('↓','')
robot_df['순이익1'] = robot_df['순이익1'].str.replace('↓','')
robot_df['순이익2'] = robot_df['순이익2'].str.replace('↓','')
robot_df['순이익3'] = robot_df['순이익3'].str.replace('↓','')

In [19]:
robot_df['매출액1'] = robot_df['매출액1'].fillna(0).apply(convert_currency_to_int)
robot_df['매출액2'] = robot_df['매출액2'].fillna(0).apply(convert_currency_to_int)
robot_df['매출액3'] = robot_df['매출액3'].fillna(0).apply(convert_currency_to_int)


robot_df['순이익1'] = robot_df['순이익1'].fillna(0).apply(convert_currency_to_int)
robot_df['순이익2'] = robot_df['순이익2'].fillna(0).apply(convert_currency_to_int)
robot_df['순이익3'] = robot_df['순이익3'].fillna(0).apply(convert_currency_to_int)

In [20]:
robot_df['매출액'] =  robot_df['매출액'].str.split('(').str[0]
robot_df['매출액'] = robot_df['매출액'].fillna(0).apply(convert_currency_to_int)
robot_df['기준년도'] = robot_df['자본총계1'].str.extract(r'(\d{4})', expand=False)

In [21]:
robot_df['자본총계1'] = robot_df['자본총계1'].str.replace('2022', '')
robot_df['자본총계2'] = robot_df['자본총계2'].str.replace('2021', '')
robot_df['자본총계3'] = robot_df['자본총계3'].str.replace('2020', '')
robot_df['자본총계1'] = robot_df['자본총계1'].str.replace('2016', '')
robot_df['자본총계2'] = robot_df['자본총계2'].str.replace('2015', '')
robot_df['자본총계3'] = robot_df['자본총계3'].str.replace('2014', '')

In [22]:
robot_df['자본총계1'] = robot_df['자본총계1'].fillna(0).apply(convert_currency_to_int)
robot_df['자본총계2'] = robot_df['자본총계2'].fillna(0).apply(convert_currency_to_int)
robot_df['자본총계3'] = robot_df['자본총계3'].fillna(0).apply(convert_currency_to_int)

In [23]:
robot_df['기준년도']= robot_df['기준년도'].fillna(0).astype(int)

In [24]:
robot_df['평균연봉']= robot_df['평균연봉'].fillna(0).astype(int)

In [25]:
robot_df.rename(columns={'회사명':'기업이름'},inplace=True)

In [26]:
unique_companies = robot_df.drop_duplicates(subset='기업이름')

In [27]:
unique_companies.reset_index(drop=True,inplace=True)

In [28]:
unique_companies.to_csv('../../data/eda_project_data/robot_rev_syt.csv',index=False)